# Imports

In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import re
from unicodedata import normalize # https://towardsdatascience.com/difference-between-nfd-nfc-nfkd-and-nfkc-explained-with-python-code-e2631f96ae6c
import pickle
from dataclasses import dataclass, field

/var/folders/pv/4lz7c47j4d901p0hrt6r1s500000gn/T/ipykernel_20369/446625484.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Load starting data 

In [51]:
file = 'data/PORTUGUESE_BRAZIL.jsonl'
data = pd.read_json(file, lines=True)

# Explore data

In [4]:
data.head()

,id,language,name,shortTitle,text,teaser,shortTeaser,kicker,regions,keywords,...,contentDate,relatedAutoTopics,contentLinks,articles,isOpinion,contentAssociations,mainContentImageLink,images,externalLinks,topStory
0,54006424,PORTUGUESE_BRAZIL,"Covid-19, racismo e mau futebol derrubam presi...",Escândalos derrubam presidente do FC Schalke 04,<p>O presidente do FC Schalke 04 e proprietári...,Após sucessivas controvérsias e péssima campan...,"Proprietário do frigorífico Tönnies, centro de...",Alemanha,[],"[{'name': 'Tönnies'}, {'name': 'covid-19'}, {'...",...,2020-07-01T00:37:53.785Z,"[{'name': 'CPI da Pandemia'}, {'name': 'Covid-...","[{'name': 'Schalke 04, o declínio de uma lenda...","[{'name': 'Schalke 04, o declínio de uma lenda...",False,"[{'id': 53326534}, {'id': 53326552}, {'id': 53...","{'description': 'Clemens Tönnies, proprietário...","[{'name': 'S04 Schalke-Boss Clemens Tönnies'},...",[],False
1,54003004,PORTUGUESE_BRAZIL,Opinião: É o momento de Merkel definir o futur...,É o momento de Merkel definir o futuro da UE,"<figure class=""placeholder-image master_landsc...",Alemanha assume presidência rotativa da UE em ...,Enquanto se prepara para entregar o cargo em 2...,Opinião,[],"[{'name': 'Angela Merkel'}, {'name': 'União Eu...",...,2020-07-01T08:42:23.316Z,"[{'name': 'Angela Merkel'}, {'name': 'Comissão...","[{'name': ''}, {'name': 'Para Merkel, UE virou...","[{'name': 'Para Merkel, UE virou questão de co...",True,"[{'id': 53322481}, {'id': 53322886}, {'id': 53...","{'description': '""Este é o maior desafio que a...",[{'name': 'Brüssel | EU Gipfeltreffen: Angela ...,[],False
2,54009380,PORTUGUESE_BRAZIL,Telescópio espacial Hubble completa 30 anos,Telescópio espacial Hubble completa 30 anos,"<p>Na prática de exercícios físicos, o que é m...","James Webb, o telescópio sucessor, deve ficar ...","James Webb, o telescópio sucessor, deve ficar ...",,[],"[{'name': 'Hubble'}, {'name': 'telescópio'}, {...",...,2020-07-01T09:20:47.843Z,[{'name': 'Telescópio Hubble'}],[{'name': 'Hubble Weltraumteleskop'}],[],False,"[{'id': 53329407}, {'id': 53329408}, {'id': 53...",{'description': None},[{'name': 'Hubble Weltraumteleskop'}],[],False
3,54008153,PORTUGUESE_BRAZIL,Polícia de Hong Kong efetua primeiras prisões ...,Hong Kong efetua primeiras prisões com base em...,<p>Um homem que foi flagrado com uma bandeira ...,Protestos irrompem no território após autorida...,Protestos irrompem no território após autorida...,Ásia,[],"[{'name': 'Hong Kong'}, {'name': 'protestos em...",...,2020-07-01T09:44:01.046Z,"[{'name': 'Hong Kong'}, {'name': 'China'}]","[{'name': ''}, {'name': 'China aprova lei de s...",[{'name': 'China aprova lei de segurança nacio...,False,"[{'id': 53328442}, {'id': 53328443}, {'id': 53...",{'description': 'Polícia prende homem em prote...,[{'name': 'Hongkong | Pro-Democracy Proteste a...,[],False
4,54009331,PORTUGUESE_BRAZIL,Trump aprova plano para retirar 9.500 soldados...,Trump aprova plano para retirar 9.500 soldados...,"<p>O presidente dos Estados Unidos, Donald Tru...",Plano ainda precisa ser apresentado ao Congres...,Plano ainda precisa ser apresentado ao Congres...,Mundo,[],"[{'name': 'Alemanha'}, {'name': 'Otan'}, {'nam...",...,2020-07-01T10:22:04.158Z,"[{'name': 'Otan'}, {'name': 'Estados Unidos'},...","[{'name': 'O mês de junho em imagens'}, {'name...","[{'name': 'Militares americanos na Alemanha, u...",False,"[{'id': 53329506}, {'id': 53329508}, {'id': 53...",{'description': 'Base americana na Alemanha. T...,[{'name': 'Aus für US-Militär-Standorte Schwei...,[],False


In [5]:
data.iloc[0]

id                                                                54006424
language                                                 PORTUGUESE_BRAZIL
name                     Covid-19, racismo e mau futebol derrubam presi...
shortTitle                 Escândalos derrubam presidente do FC Schalke 04
text                     <p>O presidente do FC Schalke 04 e proprietári...
teaser                   Após sucessivas controvérsias e péssima campan...
shortTeaser              Proprietário do frigorífico Tönnies, centro de...
kicker                                                            Alemanha
regions                                                                 []
keywords                 [{'name': 'Tönnies'}, {'name': 'covid-19'}, {'...
keywordStrings           [Tönnies, covid-19, coronavírus, pandemia, FC ...
thematicFocusCategory                                  {'name': 'Futebol'}
navigations                   [{'name': 'Alemanha'}, {'name': 'NOTÍCIAS'}]
categories               

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11848 entries, 0 to 11847
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     11848 non-null  int64 
 1   language               11848 non-null  object
 2   name                   11848 non-null  object
 3   shortTitle             11848 non-null  object
 4   text                   11848 non-null  object
 5   teaser                 11848 non-null  object
 6   shortTeaser            11848 non-null  object
 7   kicker                 11787 non-null  object
 8   regions                11848 non-null  object
 9   keywords               11848 non-null  object
 10  keywordStrings         11848 non-null  object
 11  thematicFocusCategory  11837 non-null  object
 12  navigations            11848 non-null  object
 13  categories             11848 non-null  object
 14  departments            11848 non-null  object
 15  firstPublicationDat

## kicker

Overview

In [94]:
data.kicker

0           Alemanha
1            Opinião
2                   
3               Ásia
4              Mundo
            ...     
11843       Alemanha
11844          Mundo
11845         Brasil
11846    Coronavírus
11847           None
Name: kicker, Length: 11848, dtype: object

Unique values

In [7]:
print(f'Number of unique values = {data.kicker.unique().size}')
data.kicker.value_counts()

Number of unique values = 447


Coronavírus          2283
Brasil               1484
Mundo                1063
Alemanha              837
Europa                493
                     ... 
Coronavíus              1
Sudão                   1
Bósnia                  1
guerra na Ucrânia       1
Agronegócio             1
Name: kicker, Length: 446, dtype: int64

## Thematic Focus Category

Overview

In [8]:
data.thematicFocusCategory

0         {'name': 'Futebol'}
1        {'name': 'Política'}
2         {'name': 'Ciência'}
3        {'name': 'Política'}
4        {'name': 'Política'}
                 ...         
11843     {'name': 'Cultura'}
11844    {'name': 'Política'}
11845    {'name': 'Política'}
11846       {'name': 'Saúde'}
11847       {'name': 'Saúde'}
Name: thematicFocusCategory, Length: 11848, dtype: object

Extract the significant values from the column `thematicFocusCategory`

In [9]:
thematicFocusCategory_keys = []
thematicFocusCategory_values = []
for irow,row in enumerate(data.itertuples()):
    d = row.thematicFocusCategory
    if d is not None:
        for (k,v) in d.items():
            thematicFocusCategory_keys.append(k)
            thematicFocusCategory_values.append(v)        

Unique keys in the dictionary

In [10]:
print(f'Number of unique values = {np.unique(thematicFocusCategory_keys).size}')
pd.Series(thematicFocusCategory_keys).value_counts()

Number of unique values = 1


name    11837
dtype: int64

Unique values in the dictionary

In [11]:
print(f'Number of unique values = {np.unique(thematicFocusCategory_values).size}')
pd.Series(thematicFocusCategory_values).value_counts()

Number of unique values = 43


Política                    3780
Saúde                       2936
Sociedade                    866
Conflitos                    679
Economia                     632
Natureza e meio ambiente     584
Leis e Justiça               304
Ciência                      294
Cultura                      249
História                     164
Criminalidade                158
Futebol                      148
Esporte                      136
Direitos humanos             106
Educação                      80
Religião                      77
Catástrofe                    76
Migração                      70
Tecnologia                    68
Igualdade de direitos         65
Estado de Direito             45
Liberdade de imprensa         43
Terrorismo                    39
Música                        39
Filme                         28
Variedades                    26
Literatura                    22
Segurança alimentar           22
Arte                          17
Viagem                        14
Mídia     

## Navigations

Overview

In [12]:
data.navigations

0        [{'name': 'Alemanha'}, {'name': 'NOTÍCIAS'}]
1                                 [{'name': 'Mundo'}]
2                              [{'name': 'NOTÍCIAS'}]
3                                 [{'name': 'Mundo'}]
4           [{'name': 'Mundo'}, {'name': 'Alemanha'}]
                             ...                     
11843                          [{'name': 'Alemanha'}]
11844                             [{'name': 'Mundo'}]
11845                            [{'name': 'Brasil'}]
11846                            [{'name': 'Brasil'}]
11847                          [{'name': 'NOTÍCIAS'}]
Name: navigations, Length: 11848, dtype: object

Extract the significant values from the column `navigations`

In [13]:
navigations_keys = []
navigations_values = []
for irow,row in enumerate(data.itertuples()):
    for d in row.navigations:
        if d is not None:
            for (k,v) in d.items():
                navigations_keys.append(k)
                navigations_values.append(v)        

Unique keys in the dictionary

In [14]:
print(f'Number of unique values = {np.unique(navigations_keys).size}')
pd.Series(navigations_keys).value_counts()

Number of unique values = 1


name    13235
dtype: int64

Unique values in the dictionary

In [15]:
print(f'Number of unique values = {np.unique(navigations_values).size}')
pd.Series(navigations_values).value_counts()

Number of unique values = 32


Mundo                     4570
Brasil                    2949
Alemanha                  1543
Ciência e Saúde            954
Economia                   485
NOTÍCIAS                   405
Colunas                    365
Europa                     364
Cultura                    356
Política                   295
Esporte                    267
América Latina             250
Meio Ambiente              176
NRS-Import                  89
Eleições nos EUA            71
História                    34
Turismo                     14
Especial                    12
Segunda Guerra Mundial       9
dw.com Beta                  5
Reunificação alemã           5
Calendário Histórico         3
BAKU-PT                      2
Jogos Olímpicos              2
Camarote.21                  2
Futurando!                   2
Redes Sociais                1
DW no Celular                1
Cursos de alemão             1
Eleição na Alemanha          1
ABOUT DW                     1
EXPLORE A DW                 1
dtype: i

## Categories

Overview

In [16]:
data.categories

0            [{'name': 'Futebol'}, {'name': 'Saúde'}]
1        [{'name': 'Política'}, {'name': 'Economia'}]
2                               [{'name': 'Ciência'}]
3                              [{'name': 'Política'}]
4                              [{'name': 'Política'}]
                             ...                     
11843                           [{'name': 'Cultura'}]
11844                          [{'name': 'Política'}]
11845                          [{'name': 'Política'}]
11846                             [{'name': 'Saúde'}]
11847       [{'name': 'Saúde'}, {'name': 'Política'}]
Name: categories, Length: 11848, dtype: object

Extract the significant values from the column `categories`

In [17]:
categories_keys = []
categories_values = []
for irow,row in enumerate(data.itertuples()):
    for d in row.categories:
        if d is not None:
            for (k,v) in d.items():
                categories_keys.append(k)
                categories_values.append(v)        

Unique keys in the dictionary

In [18]:
print(f'Number of unique values = {np.unique(categories_keys).size}')
pd.Series(categories_keys).value_counts()

Number of unique values = 1


name    17758
dtype: int64

Unique values in the dictionary

In [19]:
print(f'Number of unique values = {np.unique(categories_values).size}')
pd.Series(categories_values).value_counts()

Number of unique values = 46


Política                    5202
Saúde                       3544
Sociedade                   1690
Economia                    1016
Conflitos                    906
Natureza e meio ambiente     792
Leis e Justiça               648
Ciência                      512
Cultura                      380
Direitos humanos             321
História                     293
Estado de Direito            272
Criminalidade                264
Esporte                      203
Igualdade de direitos        199
Futebol                      178
Tecnologia                   154
Religião                     128
Variedades                   122
Educação                     106
Catástrofe                   100
Migração                      96
Liberdade de imprensa         89
Liberdade de expressão        61
Terrorismo                    58
Música                        48
Arte                          39
Literatura                    36
Filme                         36
Viagem                        35
Segurança 

## Departments

Overview

In [20]:
data.departments

0        [{'name': 'Brasilianisch'}]
1        [{'name': 'Brasilianisch'}]
2        [{'name': 'Brasilianisch'}]
3        [{'name': 'Brasilianisch'}]
4        [{'name': 'Brasilianisch'}]
                    ...             
11843    [{'name': 'Brasilianisch'}]
11844    [{'name': 'Brasilianisch'}]
11845    [{'name': 'Brasilianisch'}]
11846    [{'name': 'Brasilianisch'}]
11847    [{'name': 'Brasilianisch'}]
Name: departments, Length: 11848, dtype: object

Extract the significant values from the column `departments`

In [21]:
departments_keys = []
departments_values = []
for irow,row in enumerate(data.itertuples()):
    for d in row.departments:
        if d is not None:
            for (k,v) in d.items():
                departments_keys.append(k)
                departments_values.append(v)        

Unique keys in the dictionary

In [22]:
print(f'Number of unique values = {np.unique(departments_keys).size}')
pd.Series(departments_keys).value_counts()

Number of unique values = 1


name    10518
dtype: int64

Unique values in the dictionary

In [23]:
print(f'Number of unique values = {np.unique(departments_values).size}')
pd.Series(departments_values).value_counts()

Number of unique values = 3


Brasilianisch        10515
Bildungsprogramme        2
Testredaktion            1
dtype: int64

## First Publication Date

Overview

In [24]:
data.firstPublicationDate

0        2020-07-01T00:32:15.020Z
1        2020-07-01T08:42:23.316Z
2        2020-07-01T09:20:47.843Z
3        2020-07-01T09:44:01.046Z
4        2020-07-01T10:22:04.158Z
                   ...           
11843    2020-06-30T19:42:13.758Z
11844    2020-06-30T20:05:06.610Z
11845    2020-06-30T20:38:26.413Z
11846    2020-06-30T22:25:40.885Z
11847    2020-06-30T22:56:07.943Z
Name: firstPublicationDate, Length: 11848, dtype: object

In [25]:
data.firstPublicationDate.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11848 entries, 0 to 11847
Series name: firstPublicationDate
Non-Null Count  Dtype 
--------------  ----- 
11848 non-null  object
dtypes: object(1)
memory usage: 92.7+ KB


First time

In [70]:
data.firstPublicationDate.astype('datetime64[ns]').min()

Timestamp('2020-01-01 09:58:44.825000')

Last time

In [71]:
data.firstPublicationDate.astype('datetime64[ns]').max()

Timestamp('2022-07-13 20:12:54.406000')

## Last Modified Date

Overview

In [36]:
data.lastModifiedDate

0        2020-07-07T16:44:41.762Z
1        2020-07-01T10:57:00.218Z
2        2020-07-01T09:20:47.843Z
3        2020-07-01T12:09:30.928Z
4        2020-07-07T16:37:08.872Z
                   ...           
11843    2020-07-07T16:49:47.832Z
11844    2020-07-07T16:46:58.199Z
11845    2020-06-30T20:52:52.002Z
11846    2020-07-01T08:49:48.075Z
11847    2020-06-30T22:56:07.943Z
Name: lastModifiedDate, Length: 11848, dtype: object

In [32]:
data.lastModifiedDate.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11848 entries, 0 to 11847
Series name: lastModifiedDate
Non-Null Count  Dtype 
--------------  ----- 
11848 non-null  object
dtypes: object(1)
memory usage: 92.7+ KB


First time

In [34]:
data.lastModifiedDate.astype('datetime64[ns]').min()

Timestamp('2020-01-01 12:41:49.504000')

Last time

In [35]:
data.lastModifiedDate.astype('datetime64[ns]').max()

Timestamp('2022-07-13 20:12:54.406000')

## Content Date

Overview

In [37]:
data.contentDate

0        2020-07-01T00:37:53.785Z
1        2020-07-01T08:42:23.316Z
2        2020-07-01T09:20:47.843Z
3        2020-07-01T09:44:01.046Z
4        2020-07-01T10:22:04.158Z
                   ...           
11843    2020-06-30T19:42:13.758Z
11844    2020-06-30T20:05:06.610Z
11845    2020-06-30T20:38:26.413Z
11846    2020-06-30T22:32:06.904Z
11847    2020-06-30T22:56:07.943Z
Name: contentDate, Length: 11848, dtype: object

In [38]:
data.contentDate.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11848 entries, 0 to 11847
Series name: contentDate
Non-Null Count  Dtype 
--------------  ----- 
11848 non-null  object
dtypes: object(1)
memory usage: 92.7+ KB


First time

In [39]:
data.contentDate.astype('datetime64[ns]').min()

Timestamp('2020-01-01 09:58:44.825000')

Last time

In [40]:
data.contentDate.astype('datetime64[ns]').max()

Timestamp('2022-07-13 20:12:54.406000')

## Related Auto Topics

Overview

In [72]:
data.relatedAutoTopics

0        [{'name': 'CPI da Pandemia'}, {'name': 'Covid-...
1        [{'name': 'Angela Merkel'}, {'name': 'Comissão...
2                          [{'name': 'Telescópio Hubble'}]
3               [{'name': 'Hong Kong'}, {'name': 'China'}]
4        [{'name': 'Otan'}, {'name': 'Estados Unidos'},...
                               ...                        
11843    [{'name': 'Berlim'}, {'name': 'Nazismo'}, {'na...
11844    [{'name': 'ONU'}, {'name': 'Estados Unidos'}, ...
11845    [{'name': 'Jair Bolsonaro'}, {'name': 'CPI da ...
11846    [{'name': 'Sars-Cov-2'}, {'name': 'CPI da Pand...
11847                                                   []
Name: relatedAutoTopics, Length: 11848, dtype: object

Extract the significant values from the column `relatedAutoTopics`

In [41]:
relatedAutoTopics_keys = []
relatedAutoTopics_values = []
for irow,row in enumerate(data.itertuples()):
    for d in row.relatedAutoTopics:
        if d is not None:
            for (k,v) in d.items():
                relatedAutoTopics_keys.append(k)
                relatedAutoTopics_values.append(v)        

Unique keys in the dictionary

In [43]:
print(f'Number of unique values = {np.unique(relatedAutoTopics_keys).size}')
pd.Series(relatedAutoTopics_keys).value_counts()

Number of unique values = 1


name    39160
dtype: int64

Unique values in the dictionary

In [44]:
print(f'Number of unique values = {np.unique(relatedAutoTopics_values).size}')
pd.Series(relatedAutoTopics_values).value_counts()

Number of unique values = 793


CPI da Pandemia         4411
Covid-19                4104
Jair Bolsonaro          1241
Guerra na Ucrânia        988
Vladimir Putin           955
                        ... 
Dengue                     1
Calendário Histórico       1
David Bowie                1
Michelle Obama             1
François Fillon            1
Length: 793, dtype: int64

## Is Opinion

Overview

In [82]:
data.isOpinion

0        False
1         True
2        False
3        False
4        False
         ...  
11843    False
11844    False
11845    False
11846    False
11847    False
Name: isOpinion, Length: 11848, dtype: bool

Extract the significant values from the column `isOpinion`

In [46]:
data.isOpinion.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11848 entries, 0 to 11847
Series name: isOpinion
Non-Null Count  Dtype
--------------  -----
11848 non-null  bool 
dtypes: bool(1)
memory usage: 11.7 KB


Unique values

In [83]:
data.isOpinion.value_counts()

False    11199
True       649
Name: isOpinion, dtype: int64

## Images

Overview

In [85]:
data.images

0        [{'name': 'S04 Schalke-Boss Clemens Tönnies'},...
1        [{'name': 'Brüssel | EU Gipfeltreffen: Angela ...
2                    [{'name': 'Hubble Weltraumteleskop'}]
3        [{'name': 'Hongkong | Pro-Democracy Proteste a...
4        [{'name': 'Aus für US-Militär-Standorte Schwei...
                               ...                        
11843    [{'name': 'Ausstellung Enthüllt. Berlin und se...
11844             [{'name': 'Hongkong Sicherheitsgesetz'}]
11845      [{'name': 'Carlos Alberto Decotelli da Silva'}]
11846                [{'name': 'Coronavirus I Brasilien'}]
11847                                                   []
Name: images, Length: 11848, dtype: object

Extract the significant values from the column `images`

In [47]:
images = []
for irow,row in enumerate(data.itertuples()):
    images.append(len(row.images))

In [49]:
print(f'Number of unique values = {np.unique(images).size}')
pd.Series(images).value_counts()

Number of unique values = 15


1     8114
2     1916
3     1001
4      418
5      140
0      118
6       71
8       21
7       19
10      10
9       10
12       4
14       3
11       2
26       1
dtype: int64

In [92]:
np.unique(images).size

15

In [93]:
pd.Series(images).value_counts()

1     8114
2     1916
3     1001
4      418
5      140
0      118
6       71
8       21
7       19
10      10
9       10
12       4
14       3
11       2
26       1
dtype: int64

# Clean of one article

In [27]:
def normalize_white_space(text):
    """Normalize white space in a string."""
    return re.sub(r'\s+', ' ', text).strip()

def remove_puntuaction_marks(text): # Not used
    """Remove punctuation marks from a string."""
    return re.sub(r'\.|\,|\?|\!|\;|\:', '', text)

def has_space(text):
    """Check if a string has space."""
    return ' ' in text

@dataclass(frozen=True)
class Article:
    """Dataclass to easily move and operate relevant content of an article"""
    id: int
    language: str
    sentences: list[str] = field(repr=False)
    original_text: str = field(repr=False)
    number_of_sentences: int = field(init=False)
    persons: tuple[list] = field(init=False, repr=False)
    
    def __post_init__(self):
        object.__setattr__(self, 'number_of_sentences', len(self.sentences))
        object.__setattr__(self, 'persons', tuple([] for i in range(self.number_of_sentences))) # one list per sentence
    
    def render_html(self):
        """Print attribute `original_text` rendered in HTML"""
        display(HTML(self.original_text))

## Try with a single article first.

In [28]:
article = data.iloc[0]

## Explore

Text

In [29]:
article.text

'<p>O presidente do FC Schalke 04 e proprietário de um <a href="/pt-br/surto-em-frigorífico-provoca-novo-isolamento-na-alemanha/a-53913392" class="internal-link">frigorífico no centro de um novo surto de coronavírus</a>&nbsp; na Alemanha, Clemens Tönnies, renunciou nesta terça-feira (30/06) ao cargo que ocupava há 19 anos no tradicional clube de futebol alemão após se envolver em uma série de controvérsias, ainda que em questões bastante diferentes.</p>\n\n<p>O Schalke não conseguiu nenhuma vitória nos últimos 16 jogos que disputou pela Bundesliga, o campeonato alemão, na mais longa série sem um triunfo sequer na história da competição, e ainda foi goleado por 4 a 0 na última partida que disputou, contra o&nbsp;SC Freiburg.&nbsp;</p>\n\n<p>A equipe dos "azuis-reais" terminou o campeonato na 12ª colocação e não conseguiu se classificar para a Liga da Europa, o segundo mais importante torneio de clubes do continente, e ficou ainda mais longe da cobiçada vaga para a Liga dos Campeões da U

HTML

In [30]:
display(HTML(article.text))

## Clean

In [31]:
# parse HTML format of the original text
html_text = article.text
soup = BeautifulSoup(html_text, 'html.parser')

In [32]:
# delete unwanted tags like figure captions
for content in soup(['figure']):
    content.decompose()

In [33]:
# regex to filter standard end of paragraph
pattern1 = '\.$' # sentence ends with .
pattern2 = '\."$' # sentence ends with ." (quotes)
pattern3 = "\.'$" # sentence ends with .' (quotes)
pattern = '|'.join([pattern1,pattern2,pattern3]) # use or-expression to combine all 

In [34]:
# Find paragraphs in the article (original text)
paragraphs = []
for content in soup.contents:
    paragraph = content.get_text() # Convert to unicode text
    paragraph = paragraph.strip() # Strip surrounding spaces
    if re.search(pattern, paragraph):
        paragraph = normalize("NFKD", paragraph) # Unicode normalization. Eg, converts "\xa0" to a long space
        paragraph = normalize_white_space(paragraph)
        paragraph = paragraph.strip() # Strip potential surrounding spaces introduced by the "\xa0" conversion
        paragraphs.append(paragraph)

In [35]:
# Find sentences in the paragraphs
sentences = []
for paragraph in paragraphs:
    sentences.extend(re.split('(?<=\.) ', paragraph)) # split text when ". "

In [36]:
sentences = list(filter(has_space, sentences)) # Needed to classify as sentence

Check results

In [37]:
sentences

['O presidente do FC Schalke 04 e proprietário de um frigorífico no centro de um novo surto de coronavírus na Alemanha, Clemens Tönnies, renunciou nesta terça-feira (30/06) ao cargo que ocupava há 19 anos no tradicional clube de futebol alemão após se envolver em uma série de controvérsias, ainda que em questões bastante diferentes.',
 'O Schalke não conseguiu nenhuma vitória nos últimos 16 jogos que disputou pela Bundesliga, o campeonato alemão, na mais longa série sem um triunfo sequer na história da competição, e ainda foi goleado por 4 a 0 na última partida que disputou, contra o SC Freiburg.',
 'A equipe dos "azuis-reais" terminou o campeonato na 12a colocação e não conseguiu se classificar para a Liga da Europa, o segundo mais importante torneio de clubes do continente, e ficou ainda mais longe da cobiçada vaga para a Liga dos Campeões da Uefa.',
 'A participação em uma dessas grandes competições europeias seria uma importante fonte de renda para a eq

## Collect

Original

In [38]:
article

id                                                                54006424
language                                                 PORTUGUESE_BRAZIL
name                     Covid-19, racismo e mau futebol derrubam presi...
shortTitle                 Escândalos derrubam presidente do FC Schalke 04
text                     <p>O presidente do FC Schalke 04 e proprietári...
teaser                   Após sucessivas controvérsias e péssima campan...
shortTeaser              Proprietário do frigorífico Tönnies, centro de...
kicker                                                            Alemanha
regions                                                                 []
keywords                 [{'name': 'Tönnies'}, {'name': 'covid-19'}, {'...
keywordStrings           [Tönnies, covid-19, coronavírus, pandemia, FC ...
thematicFocusCategory                                  {'name': 'Futebol'}
navigations                   [{'name': 'Alemanha'}, {'name': 'NOTÍCIAS'}]
categories               

Dataclass

In [39]:
article = Article(article.id, article.language, sentences, article.text)

In [40]:
article

Article(id=54006424, language='PORTUGUESE_BRAZIL', number_of_sentences=41)

Take a look at the sentences we found

In [41]:
article.sentences

['O presidente do FC Schalke 04 e proprietário de um frigorífico no centro de um novo surto de coronavírus na Alemanha, Clemens Tönnies, renunciou nesta terça-feira (30/06) ao cargo que ocupava há 19 anos no tradicional clube de futebol alemão após se envolver em uma série de controvérsias, ainda que em questões bastante diferentes.',
 'O Schalke não conseguiu nenhuma vitória nos últimos 16 jogos que disputou pela Bundesliga, o campeonato alemão, na mais longa série sem um triunfo sequer na história da competição, e ainda foi goleado por 4 a 0 na última partida que disputou, contra o SC Freiburg.',
 'A equipe dos "azuis-reais" terminou o campeonato na 12a colocação e não conseguiu se classificar para a Liga da Europa, o segundo mais importante torneio de clubes do continente, e ficou ainda mais longe da cobiçada vaga para a Liga dos Campeões da Uefa.',
 'A participação em uma dessas grandes competições europeias seria uma importante fonte de renda para a eq

Take a look at empty lists (one per sentences) that will be later filled in with the persons' names using the NER. 

In [30]:
article.persons

([],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [])

Check if the lenths agree with each other

In [42]:
print(len(article.sentences), len(article.persons))

41 41


# Cleaning of all articles

In [43]:
articles = []

for article in data.itertuples(index=False):
    
    html_text = article.text
    soup = BeautifulSoup(html_text, 'html.parser')

    # delete unwanted tags like figure captions
    for content in soup(['figure']):
        content.decompose()

    # regex to filter standard end of paragraph
    pattern1 = '\.$' # sentence ends with .
    pattern2 = '\."$' # sentence ends with ." (quotes)
    pattern3 = "\.'$" # sentence ends with .' (quotes)
    pattern = '|'.join([pattern1,pattern2,pattern3]) # use or-expression to combine all 

    paragraphs = []
    for content in soup.contents:
        paragraph = content.get_text() # Convert to unicode text
        paragraph = paragraph.strip() # Strip surrounding spaces
        if re.search(pattern, paragraph):
            paragraph = normalize("NFKD", paragraph) # Unicode normalization. Eg, converts "\xa0" to a long space
            paragraph = normalize_white_space(paragraph)
            paragraph = paragraph.strip() # Strip potential surrounding spaces introduced by the "\xa0" conversion
            paragraphs.append(paragraph)

    sentences = []
    for paragraph in paragraphs:
        sentences.extend(re.split('(?<=\.) ', paragraph)) # split text when ". "

    sentences = list(filter(has_space, sentences)) # Needed to classify as sentence

    articles.append(Article(article.id, article.language, sentences, article.text))

## Check results

Number of articles processed

In [44]:
len(articles)

11848

Display info about the first 10 articles

In [45]:
articles[:10]

[Article(id=54006424, language='PORTUGUESE_BRAZIL', number_of_sentences=41),
 Article(id=54003004, language='PORTUGUESE_BRAZIL', number_of_sentences=34),
 Article(id=54009380, language='PORTUGUESE_BRAZIL', number_of_sentences=14),
 Article(id=54008153, language='PORTUGUESE_BRAZIL', number_of_sentences=22),
 Article(id=54009331, language='PORTUGUESE_BRAZIL', number_of_sentences=20),
 Article(id=54010979, language='PORTUGUESE_BRAZIL', number_of_sentences=32),
 Article(id=54011278, language='PORTUGUESE_BRAZIL', number_of_sentences=19),
 Article(id=54012392, language='PORTUGUESE_BRAZIL', number_of_sentences=161),
 Article(id=54015488, language='PORTUGUESE_BRAZIL', number_of_sentences=13),
 Article(id=54015413, language='PORTUGUESE_BRAZIL', number_of_sentences=19)]

### Check article 0

In [46]:
articles[0].render_html()

In [47]:
articles[0].sentences[0]

'O presidente do FC Schalke 04 e proprietário de um frigorífico no centro de um novo surto de coronavírus na Alemanha, Clemens Tönnies, renunciou nesta terça-feira (30/06) ao cargo que ocupava há 19 anos no tradicional clube de futebol alemão após se envolver em uma série de controvérsias, ainda que em questões bastante diferentes.'

In [16]:
articles[0].sentences[1]

'O Schalke não conseguiu nenhuma vitória nos últimos 16 jogos que disputou pela Bundesliga, o campeonato alemão, na mais longa série sem um triunfo sequer na história da competição, e ainda foi goleado por 4 a 0 na última partida que disputou, contra o SC Freiburg.'

In [17]:
articles[0].sentences[-1]

'Ele, porém, garante que vai tirar o frigorífico da crise.'

In [18]:
articles[0].sentences[-2]

'A revolta gerada pelo papel da empresa no ressurgimento do coronavírus na região, após a doença estar amplamente controlada, levou muitas pessoas a pedirem o afastamento de Clemens Tönnies da companhia.'

In [39]:
articles[0].sentences[10]

'Na época, o clube, porém, afirmou que Tönnies decidiu se afastar da entidade por um período de três meses.'

In [40]:
articles[0].sentences[20]

'Estarei ligado ao FC Schalke 04 por toda a minha vida", afirmou Tönnies em seu pedido de desligamento.'

### Check article 1

In [116]:
articles[1].render_html()

In [19]:
articles[1].sentences[0]

'Em meados de março, quando o coronavírus começou a se espalhar pela Europa, a chanceler federal alemã, Angela Merkel, deu um aviso implacável: "Este é o maior desafio que a União Europeia já enfrentou", disse.'

In [20]:
articles[1].sentences[1]

'Agora, à medida que os países-membros começam a relaxar o isolamento e reabrem as fronteiras, a atenção se volta à recuperação.'

In [43]:
articles[1].sentences[2]

'Se existe algum país em condições de catalisar a integração fiscal necessária para estabilizar a Europa, esse país é a Alemanha.'

In [21]:
articles[1].sentences[-1]

'"Ela será construída com realizações concretas".'

In [22]:
articles[1].sentences[-2]

'"A Europa não será feita de uma só vez, ou de acordo com um único plano", disse Schuman em maio de 1950.'

In [46]:
articles[1].sentences[10]

'O que não é apenas incomum, mas inovador, é a proposta dos dois líderes para financiar o fundo, permitindo que a Comissão Europeia contraia empréstimos nos mercados financeiros – um passo anteriormente impensável em direção à dívida comum.'

In [48]:
articles[1].sentences[20]

'Hamilton previu que estabeleceria as bases para uma identidade fiscal coletiva, fortalecendo a nova união não apenas no país, mas também no exterior, tornando seus mercados e seus títulos mais atraentes para os investidores do que a dívida de qualquer estado individual.'

# Save results

In [50]:
outputdir = Path('data')
file = 'portuguese_sentences.pickle'
with open(outputdir/file, 'wb') as file:
    pickle.dump(articles, file)